# Cleaning up the doelpunten.csv file

In [3]:
import pandas as pd

### Creating a dataframe

In [4]:
df = pd.read_csv("doelpunten.csv", encoding='latin1', 
names=['Match_ID', 'Seizoen', 'Speeldag', 'Datum', 'Startuur', 'Thuisploeg', 'Uitploeg', 
'Scorende ploeg', 'Tijdstip goal', 'Score thuisploeg', 'Score Uitploeg'])

#### Adding column for minute of goal

In [5]:
# tijdstip doelpunt in geheel getal
def convert(row):
    uur, minuut = row['Tijdstip goal'].split(":")
    start_uur, start_minuut = row['Startuur'].split(":")
    
    uur = int(uur)
    minuut = int(minuut)
    start_uur = int(start_uur)
    start_minuut = int(start_minuut)

    if uur < start_uur:
        absolute_minuten = (60 - start_minuut) + (uur + 1 - start_uur) * 60 + minuut
    else:
        absolute_minuten = (uur - start_uur) * 60 + minuut - start_minuut
    
    return absolute_minuten

In [6]:
df['Minuut goal'] = df.apply(convert, axis=1)

In [7]:
controle_binnen_tijd = df[df['Minuut goal'] > 120]
print(len(controle_binnen_tijd))

0


### Creating second dataframe

In [8]:
wedstrijden_df = pd.read_csv('wedstrijden.csv', encoding='latin1',
names=['Match_ID', 'Seizoen', 'Speeldag', 'Datum', 'Startuur', 'Thuisploeg', 
'Score thuisploeg', 'Score Uitploeg', 'Uitploeg'])

#### Merge both dataframes

In [9]:
df_unique = df.drop_duplicates(subset=['Match_ID'], keep='last')
# wedstrijden_df_unique = wedstrijden_df.drop_duplicates(subset=['Match_ID'], keep='last')

df_unique['Match_ID'] = df_unique['Match_ID'].astype(str)
wedstrijden_df.Match_ID = wedstrijden_df.Match_ID.astype(str)

merged_df = pd.merge(df_unique, wedstrijden_df, on="Match_ID", how="inner")

C:\Users\simon\AppData\Local\Temp\ipykernel_12812\268699993.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['Match_ID'] = df_unique['Match_ID'].astype(str)


#### Vergelijken van de datums

In [10]:
merged_df['Datums_gelijk'] = merged_df['Datum_x'] == merged_df['Datum_y']

merged_df["Datums_gelijk"].value_counts()

Datums_gelijk
True    151
Name: count, dtype: int64

#### Controleren of uiteindelijke resultaat overeenkomt met doelpunten

In [11]:
fouten = merged_df[(merged_df['Score thuisploeg_x'] != merged_df['Score thuisploeg_y']) | 
                        (merged_df['Score Uitploeg_x'] != merged_df['Score Uitploeg_y'])]

if len(fouten) == 0:
    print("Geen fouten")
else:
    print(fouten.index)

Index([  3,   4,   5,  12,  13,  14,  15,  16,  17,  18,  19,  20,  24,  25,
        26,  27,  28,  29,  34,  35,  36,  37,  38,  40,  41,  43,  44,  45,
        46,  48,  52,  55,  56,  57,  59,  62,  63,  64,  65,  69,  70,  71,
        78,  79,  80,  81,  82,  83,  85,  87,  88,  89,  91,  92,  94,  95,
        96,  97,  98,  99, 100, 101, 105, 106, 107, 111, 112, 113, 114, 115,
       116, 117, 118, 120, 121, 122, 123, 125, 126, 128, 129, 130, 131, 133,
       135, 136, 137, 138, 139, 140, 141, 143, 145, 146, 148],
      dtype='int64')
